# How to test optimization problems
When implementing a new model, we all make mistakes. Whether it is putting a $+$ where a $-$ should be, swapping $\leq$ with $\geq$ or adding a zero by accident. In most cases, we realize that something is wrong because we try to solve the model, and the result is infeasible when it shouldn't be or the result does not make sense. So we stare at the code until we see the problem, do a facepalm and move on.

However, the more involved a model becomes, the more difficult this gets. So if you have a complicated scheduling model for a massive supply chain, finding out where the problem is can take hours, if not days. So what can we do about this? Turns out, in the software development world they have the exact same problem, and they came up with something called [unit tests](https://www.atlassian.com/continuous-delivery/software-testing/types-of-software-testing).

Unit tests, like the name suggests, are tests that you run for a unit of code. Typically, a unit of code is a part of code that you cannot reduce any further, i.e. a modular building block for the rest of the code. The logic goes that if you test all your building blocks, then chances are that the code is correct. So how can we use this for optimization models? In this post I am going to give you a framework for how you can do this, but it requires that you follow my object-oriented coding practices that I laid out [here](https://github.com/RichardOberdieck/optimization-blog/blob/master/Linking%20object-oriented%20programming%20and%20optimization.ipynb). So in a nutshell, how do you test optimization problems?

1) Write object oriented code

2) Get small meaningful datasets for your model

3) For each constraint type: create a bunch of scenarios which create feasible or infeasible models. Solve those instances and check whether your prediciton holds.

5) For callbacks: follow the same principle, i.e. create a dummy model and only use the callback. Check result against expected result.

## Small recap - what I will be using
I'll be using Xpress, the community license for which you can get by running

```conda install -c fico-xpress xpress```

in the command prompt. You can find a short introduction to the Python API for Xpress [here](https://github.com/RichardOberdieck/optimization-blog/blob/master/The%20Xpress%20Python%20API.ipynb) As for the principles of object-oriented programming, they can be summarized as follows:

1)	Use a general, object oriented programming language

2)	Use classes for your model and indices

3)	Every variable is a property in your model class

4)	Every constraint, objective function and callback is a method in your model class

5)	There are no other methods except these in your model class. If there are, have a good reason for them.

Lastly, to show how an example on how to implement this, we'll be looking at a simple shipping problem:
\begin{equation}
\begin{array}{lll}
\underset{x}{\text{minimize}} & \sum \limits_{o,d} c_{o,d}^Tx_{o,d} & \text{Shipping cost} \\
\text{subject to} & \sum \limits_{o} x_{o,d} \geq D_d, \hspace{0.15cm} \forall d & \text{Demand satisfaction} \\
& \sum \limits_{d} x_{o,d} \leq S_o, \hspace{0.15cm} \forall o & \text{Supply limitation} \\
& 0 \leq x_{o,d} \leq S_o, \hspace{0.15cm} \forall o,d & \text{Bounds}
\end{array}
\end{equation}

In [33]:
# This is all the bookkeeping
import xpress as xp
import numpy as np
import collections

Coordinate = collections.namedtuple("Coordinate", "easting northing")
Origin = collections.namedtuple("Origin", "name coordinate supply_limit")
Destination = collections.namedtuple("Destination", "name coordinate demand")
class Connection:
    def __init__(self, origin, destination):
        self.origin = origin
        self.destination = destination
        self.distance = self.get_distance()
    
    def get_distance(self):
        return np.sqrt((self.origin.coordinate.easting - self.destination.coordinate.easting)**2 + 
                       (self.origin.coordinate.northing - self.destination.coordinate.northing)**2)

## Let's build the model
Following the pricinples laid up above, let's build the model:

In [30]:
class Modeller:
    def __init__(self, connections, shipping_cost):
        self.connections = connections
        self.shipping_cost = shipping_cost
        
        # Let's define the variables and model object
        self.x = self.get_variables()
        self.mdl = xp.problem("Shipping problem")
        self.mdl.addVariable(self.x)
        
    def get_variables(self):
        """
        This defines the variables used in the optimization.
        """
        return {c : xp.var(vartype = xp.continuous, lb = 0, ub = c.origin.supply_limit, 
                           name = f'x_{c.origin.name,c.destination.name}') for c in self.connections}
    
    def get_shipping_cost(self):
        return xp.Sum(self.shipping_cost * c.distance * self.x[c] for c in self.connections)
    
    def ensure_demand_satisfaction(self):
        demand_satisfaction = [xp.constraint(xp.Sum(self.x[c] for c in self.connections if c.destination == d) 
                                             >= d.demand, name = f'Demand for {d.name}')
                               for d in set([c.destination for c in self.connections])]
        self.mdl.addConstraint(demand_satisfaction)
        
    def ensure_supply_limit(self):
        supply_limit = [xp.constraint(xp.Sum(self.x[c] for c in self.connections if c.origin == o) <= o.supply_limit, 
                                            name = f'Supply limit {o.name}')
                               for o in set([c.origin for c in self.connections])]
        self.mdl.addConstraint(supply_limit)
        
    def build_model(self):
        self.mdl.setObjective(self.get_shipping_cost())
        self.ensure_demand_satisfaction()
        self.ensure_supply_limit()
    
    def run_model(self):
        self.build_model()
        self.mdl.solve()
        return self.get_shipping_quantities()
    
    def get_shipping_quantities(self):
        return {(c.origin.name, c.destination.name) : self.mdl.getSolution(self.x[c]) for c in self.connections}

## So how do we test now?
This looks all nice and shiny now, but how do we actually test? Well, the basic idea is this:
> We test each method by solving only using those constraints/objective function and check the expected result.

This means, we end up with a bunch of test for each method, where we throw some values in, and check whether what we get out is also what we expect. For the objective function, we expect to get out a certain value (within machine precision), while for constraints we expect to get feasibility/infeasibility. Note that we **never** test multiple methods at the same time, i.e. you don't use the objective function when testing for constraints. The reason is that this way you create a dependency that you need to maintain forever.

So how does this look in practice?

### The data
The first thing you need to test something is test data. So let's create a small test scenario:

In [34]:
origins = [Origin(name = "Odense", coordinate = Coordinate(587760, 6140138), supply_limit = 110),
          Origin(name = "Kolding", coordinate = Coordinate(530680, 6149322), supply_limit = 200)]
destinations = [Destination(name = "Copenhagen", coordinate = Coordinate(347017, 6173983), demand = 100),
               Destination(name = "Hamburg", coordinate = Coordinate(566407, 5933904), demand = 150)]
connections = [Connection(o,d) for o in origins for d in destinations]
shipping_cost = 50 # Per km and unit shipped

### The test script
The second thing you need is a small script that evaluates to `True` or `False`, and to which you know the answer to. Let's write a small example for the demand satisfaction:
\begin{equation}
\sum \limits_{o} x_{o,d} \geq D_d
\end{equation}

In [35]:
def test_demand_satisfaction(connections):
    model = Modeller(connections, 0)
    model.ensure_demand_satisfaction()
    model.mdl.solve()
    return model.mdl.getProbStatus() != 2 

To check whether we implemented everything correctly, we test out different values for $D_d$ and see whether the resulting problem is feasible or not. We know that we have a total of $110+200=310$ in supply, so any $D_d > 310$ should make the problem infeasible. Let's test!

In [39]:
# The first case
test_result = test_demand_satisfaction(connections)
print(f'Test result: {test_result}')

# Put in a set of demands that violate the constraint
destinations = [Destination(name = "Copenhagen", coordinate = Coordinate(347017, 6173983), demand = 350),
               Destination(name = "Hamburg", coordinate = Coordinate(566407, 5933904), demand = 10)]
connections = [Connection(o,d) for o in origins for d in destinations]
test_result = test_demand_satisfaction(connections)
print(f'Test result: {test_result}')

Test result: True
Test result: False


## What about variables and callbacks?
Variable creation is not necessariy needed to be tested in most cases, as it is fairly straightforward. However, in some cases it may be a good idea, so just keep an eye out.

Callbacks on the other hand are really tricky, because you cannot really decouple them from the whole optimization problem. So for example if you want to introduce cuts when an optimal solution has been found, then you need a full-fletched optimization problem to work this out.

Unfortunately, as far as I know there is no way around this, meaning that you need to create the smallest possible instances of your full model and test those. I am really not happy about this, as it defeats the moduarity premise of unit tests, however if you have written unit tests for your constraints and all the other parts of your model, then you should be reasonably certain that a failure in a callback test will be related to the callback rather than a problematic constraint formulation.

## How to maintain tests?
By now I hopefully have convinced you that testing is a great idea. However, you may think now that it is really a lot of work to run these scripts manually all the time. Well, good news! There is A LOT of software out there helping you to write unit tests, running them automatically and showing whenever any of them fail.

Two resoureces I would really recommend at this point for Python are [this](https://docs.python.org/3/library/unittest.html) and [this](https://docs.python-guide.org/writing/tests/) post, which explain how you do testing. But this applies to any language - even MATLAB has a [testing framework](https://www.mathworks.com/help/matlab/class-based-unit-tests.html).

It's never to late to start writing tests. Your future self will say thank you!